#### Code to download and separate geopotential height data.

Updated: November 6, 2024

I moved files around for the purpose of making things more organized for this project. This is an old coding file that goes through and saves ERA5 data in a more manageable xarray format.

In [1]:
#import statements... I think these are all of the relevant ones to what I am doing here. 
import datetime as dt
import numpy as np
import xarray as xr
import math
import netCDF4
import os
import scipy.stats
import pickle
import pandas as pd

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


I'm going to loop through gph and temp dates to create arrays of values for the levels and whatnot that I am looking for. 

I had to separate these into two files due to their size. 

In [2]:
#year = [i for i in range(1991,2022)] #indicate years for data, needed for opening files
year = [i for i in range(1959,1991)] 
#index = [i for i in range(1,43)]

In [3]:
len(year)

32

In [4]:
#create empty arrays for the years of data
gph = np.empty((len(year),152,361,1440))
gph[:] = np.nan

In [5]:
#this is my attempt at looping through gph and combining the files lol
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959
    #if year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
    if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987:
        
        gfile1 = xr.open_dataset("../../../era5/gph/era5_gph_"+str(year[i])+".nc")
        g_files1 = gfile1["z"] 
        g_data1 = g_files1.loc[dict(level=500)]
        time_coord1 = g_data1.time.values
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../../../era5/gph/era5_gph_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["z"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=500)]
        time_coord2 = g_data2.time.values
        values2 =  g_data2.values

        #combine into np array
        gph[i,:61,:,:] = values1[90:,:,:]
        gph[i,61:,:,:] = values2[:91,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    #if year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
    if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988:
        gfile1 = xr.open_dataset("../../../era5/gph/era5_gph_"+str(year[i])+".nc")
        g_files1 = gfile1["z"] #next line will reduce this to the mean 
        g_data1 = g_files1.loc[dict(level=500)]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("../../../era5/gph/era5_gph_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["z"] #next line will reduce this to the mean
        g_data2 = g_files2.loc[dict(level=500)]
        values2 =  g_data2.values

        #combine into np array
        gph[i,:61,:,:] = values1[91:,:,:]
        gph[i,61:120,:,:] = values2[:59,:,:]
        gph[i,121:,:,:] = values2[60:91,:,:]
        continue
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("../../../era5/gph/era5_gph_"+str(year[i])+".nc")
        g_files1 = gfile1["z"] #next line will reduce this to the mean over th
        g_data1 = g_files1.loc[dict(level=500)]
        values1 =  g_data1.values

        gfile2 = xr.open_dataset("../../../era5/gph/era5_gph_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["z"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(level=500)]
        values2 =  g_data2.values
        
        gph[i,:61,:,:] = values1[90:,:,:]
        gph[i,61:120,:,:] = values2[:59,:,:]
        gph[i,121:,:,:] = values2[60:91,:,:]
        continue

1959


/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'rasterio' loading failed:
(numpy 1.24.3 (/nfs/home11/grad/2020/ef935217/.local/lib/python3.8/site-packages), Requirement.parse('numpy<1.23.0,>=1.16.5'), {'scipy'})
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990


In [6]:
gph.shape

(32, 152, 361, 1440)

In [7]:
pickle.dump(gph, open("FullNH_500_temp_1959_PT1.p", 'wb')) #download was changed because temperature
#i also did NOT divide by 10 here to conserve memory